# Indian food (Dishes?) analysis

> #### by mahesh_patapalli

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
data = pd.read_csv('../input/indian-food-101/indian_food.csv')
data.head()
data_clean = data.replace(to_replace = ['-1',-1], value = np.nan)
data_clean = data_clean.dropna().reset_index()
data_clean = data_clean.drop(['index'],axis = 1)

### let's talk about numbers first!

### Food Preparation

Food preparation can involve any number of actions, but for this lesson, we're going to organize them into three broad types of activities. First is cleaning.

Food preparation involves the entire spectrum of events - from obtaining to eating food - and most of the food that we get in raw form is not best to eat as is. So, the first step in food preparation is cleaning the food. The need for this is obvious for things like meats, which can contain harmful bacteria if not cleaned properly. In many cultures, including certain Jewish and Muslim dietary restrictions, meat is deemed unfit to eat if there's blood on it, so the cleaning stage is vital. There are reasons that cultures have such restrictions; they're based on rules for survival.

Cleaning is just as important for other ingredients as well, including natural fruits, vegetables, and grains. Most food products we consume today are not only grown commercially but are treated with pesticides or other chemicals to ensure the survival of the crop. These chemicals should be washed off of produce before consumption. Additionally, dust, natural bacteria, and trace chemicals from insects or birds can be found on organic produce.

In [ ]:
fig = px.strip(data_clean, x = 'diet', y = 'prep_time', color = 'diet', title ='Vegan and non-Vegan prep_times')
fig.show()

In [ ]:
fig = px.strip(data_clean, x = 'diet', y = 'cook_time', color = 'diet', title ='Vegan and non-Vegan cook_times')
fig.show()

In [ ]:
fig = px.pie(values = data_clean['course'].value_counts().unique(), names = data_clean['course'].value_counts().index,title = 'distribution of indian dishes based on course')
fig.show()

In [ ]:
fig = px.pie(values = data_clean['flavor_profile'].value_counts().unique(), names = data_clean['flavor_profile'].value_counts().index,title = 'distribution of indian dishes based on flavor')
fig.show()

In [ ]:
state_frame = pd.DataFrame()
for state in data_clean['state'].value_counts().index:
    state_f = data_clean[(data_clean['state'] == state)]
    if len(state_f['diet'].value_counts().values) == 1:
        state_frame = pd.concat([state_frame,pd.DataFrame([[state,state_f['diet'].value_counts().values[0],0]])],ignore_index = True)
    else:
        state_frame = pd.concat([state_frame,pd.DataFrame([[state,state_f['diet'].value_counts().values[0],state_f['diet'].value_counts().values[1]]])],ignore_index = True)
state_frame = state_frame.rename(columns = {0:'state',1:'vegetarian',2:"non-vegetarian"})

### Indian vegetarian dishes

Vegetarian cuisine is based on food that meets vegetarian standards by not including meat and animal tissue products (such as gelatin or animal-derived rennet). For lacto-ovo vegetarianism (the most common type of vegetarianism in the Western world), eggs and dairy products (such as milk and cheese without rennet) are permitted. For lacto vegetarianism, dairy products are permitted but eggs are not, and for ovo vegetarianism eggs are permitted but dairy products are not.The strictest form of vegetarianism is veganism, which excludes all animal products, including dairy, honey, and some refined sugars if filtered and whitened with bone char.There are also partial vegetarians (flexitarians), such as pescetarians who may eat fish but avoid other types of meat.

Vegetarian foods can be classified into several different types:

* Traditional foods that have always been vegetarian include cereals, grains, fruits, vegetables, legumes and nuts.
* Soy products, including tofu and tempeh, which are common protein sources.
* Textured vegetable protein (TVP), made from defatted soy flour, often included in chili and burger recipes in place of ground meat.
* Meat analogues, which mimic the taste, texture, and appearance of meat and are often used in recipes that traditionally contained meat.
* Eggs and dairy product analogues in vegan cuisine (such as aquafaba, plant cream or plant milk).

In [ ]:
fig = px.bar(state_frame,x='state',y = 'vegetarian' ,color = 'state',title = 'vegetarian dishes on states')
fig.show()

Non-vegetarian (non-veg) is an Indian English word that is used to refer to meaty food and also to a person who is a meatarian not a vegetarian i.e. someone who consumes meat, especially as a major source of protein. Although the intended meaning is readily understood, these terms are not common parlance in the Anglosphere (English-speaking countries) where meat consumption is the norm, vegetarianism is a rarity, and veganism is a novelty. In an Indian context, the term has often been criticized as a misnomer and a casteist pejorative with origins attributed to the caste system in India.

A related word is eggetarian, that refers to a vegetarian who consume egg-based products but not meat.

In [ ]:
fig = px.bar(state_frame,x='state',y = 'non-vegetarian' ,color = 'state',title = 'non - vegetarian dishes on states')
fig.show()

In [ ]:
fig = px.bar(data_clean, x="region", y='cook_time',color='diet', barmode='group',title = 'cooktimes based on region (veg/non veg)')
fig.show()

## analysis based on regions

#### ***north east*** and ***central*** do not have enough data for quality visualization

In [ ]:
wet_set = data_clean[data_clean['region'] == 'West']
east_set = data_clean[data_clean['region'] == 'East']
north_set = data_clean[data_clean['region'] == 'North']
south_set = data_clean[data_clean['region'] == 'South']

In [ ]:
def top_ingredients_regions(region_set):
    ingredients = {}
    for row in region_set.ingredients:
        for ingredient in row.split(sep = ','):
            ingredient = ingredient.replace(" ","")
            if ingredient.lower() not in ingredients.keys():
                ingredients[ingredient.lower()] = 1
            else:
                ingredients[ingredient.lower()] = ingredients[ingredient.lower()] +1
    ingredients_sorted = sorted(ingredients.items(), key=lambda x: x[1], reverse=True)[:10]
    return dict(ingredients_sorted)

### ingredients used based on regions

A food ingredient is any substance that is added to a food to achieve a desired effect. The term “food ingredient” includes food additives, which are substances added to foods for specific technical and/or functional purposes during processing, storage or packaging.

There are two types of food additives—direct and indirect. Direct food additives are used in foods to impart specific technological or functional qualities. For example, stabilizers are used to help prevent separation of nutrients in fortified milk products, while phosphates are used as a leavening agent in baked goods. Indirect additives are not intentionally added to food, but may be present in trace amounts as a result of processing, packaging, shipping or storage.

Both direct and indirect food additives are controlled by national regulatory authorities, such as the U.S. Food and Drug Administration. Any food ingredient must be proven safe to be used in foods.

In [ ]:
wet_set_dict = top_ingredients_regions(wet_set)
east_set_dict = top_ingredients_regions(east_set)
north_set_dict = top_ingredients_regions(north_set)
south_set_dict = top_ingredients_regions(south_set)

In [ ]:
fig = px.bar(x = wet_set_dict.keys(),y = wet_set_dict.values(), color = wet_set_dict.keys(),title = 'top 10 ingredients used in West-india')
fig.show()

In [ ]:
fig = px.bar(x = east_set_dict.keys(),y = east_set_dict.values(), color = east_set_dict.keys(),title = 'top 10 ingredients used in East-india')
fig.show()

In [ ]:
fig = px.bar(x = north_set_dict.keys(),y = north_set_dict.values(), color = north_set_dict.keys(),title = 'top 10 ingredients used in North-india')
fig.show()

In [ ]:
fig = px.bar(x = south_set_dict.keys(),y = south_set_dict.values(), color = south_set_dict.keys(),title = 'top 10 ingredients used in South-india')
fig.show()

## A multicourse_meal

A multicourse meal or full-course dinner is a meal of multiple courses, almost invariably eaten in the evening or afternoon. Most Western-world multicourse meals follow a standard sequence, influenced by traditional French haute cuisine. Each course is supposed to be designed with a particular size and genre that befits its place in the sequence. There are variations depending on location and custom. The following is a common sequence for multicourse meals:

The meal begins with an hors d'oeuvre or appetizer, a small serving that usually does not include red meat. In Italian custom, antipasto is served, usually finger food that does not contain pasta or any starch.
This may be followed by a variety of dishes, including a possible fish course or other light fare. The number and size of these intermittent courses is entirely dependent on local custom.
Following these is the main course. This is the most important course and is usually the largest.
Next comes the salad course, although salad may often refer to a cooked vegetable, rather than the greens most people associate with the word. Note that in America since around 1960, the salad course (usually a small, simple green salad lightly dressed) is served at some point before the main course. Sometimes, the salad also accompanies the cheese course.
The meal may carry on with a cheese selection, accompanied by an appropriate selection of wine. In many countries cheeses will be served before the meal, and in the United States often between the main course and dessert, just like in most European countries. In the UK, more typically the cheese course will follow dessert. Nuts are also a popular after-meal selection (thus the common saying "from soup to nuts", meaning from beginning to end).
The meal will often culminate with a dessert, either hot or cold, sometimes followed with a final serving of hot or cold fruit and accompanied by a suitable dessert wine.

#### starter
starter is often the first course of a meal, served directly before the main course. Starters come in a wide variety and may be served hot (vol-au-vents, soufflés) or cold (cold cuts, pâtés). However, as part of a formal dinner, they may be served after the soup or the hors d’oeuvres. In this case, the starter is therefore no longer the first dish of a meal, but comes second or third in the sequence of courses. Hors d’oeuvres are smaller dishes than starters. They consist of a varied assortment of hot or cold food served at the beginning of the meal, to stimulate the appetite, without being too rich.

#### snack
A snack is a small service of food and generally eaten between meals. Snacks come in a variety of forms including packaged snack foods and other processed foods, as well as items made from fresh ingredients at home.
Traditionally, snacks are prepared from ingredients commonly available at home without a great deal of preparation. Often cold cuts, fruits, leftovers, nuts, sandwiches, and sweets are used as snacks. With the spread of convenience stores, packaged snack foods became a significant business.
Snack foods are typically designed to be portable, quick, and satisfying. Processed snack foods, as one form of convenience food, are designed to be less perishable, more durable, and more portable than prepared foods. They often contain substantial amounts of sweeteners, preservatives, and appealing ingredients such as chocolate, peanuts, and specially-designed flavors (such as flavored potato chips).
Beverages, such as coffee, are not generally considered snacks although they may be consumed along with or in lieu of snack foods.
A snack eaten shortly before going to bed or during the night may be called a "bedtime snack", "late night snack", or "(mid)night snack".



#### main course
The main dish is usually the heaviest, heartiest, and most complex or substantial dish in a meal. The main ingredient is usually meat, fish or another protein source. It is most often preceded by an appetizer, soup or salad, and followed by a dessert. For those reasons the main course is sometimes referred to as the "meat course".

In formal dining, a well-planned main course can function as a sort of gastronomic apex or climax. In such a scheme, the preceding courses are designed to prepare for and lead up to the main course in such a way that the main course is anticipated and, when the scheme is successful, increased in its ability to satisfy and delight the diner. The courses following the main course then calm the palate and the stomach, acting as a sort of dénouement or anticlimax.

#### dessert
Dessert is a course that concludes a meal. The course usually consists of sweet foods, such as confections, and possibly a beverage such as dessert wine or liqueur. In some parts of the world, such as much of central and western Africa, and most parts of China and India, there is no tradition of a dessert course to conclude a meal.

The term dessert can apply to many confections, such as biscuits, cakes, cookies, custards, gelatins, ice creams, pastries, pies, puddings, sweet soups, and tarts. Fruit is also commonly found in dessert courses because of its naturally occurring sweetness. Some cultures sweeten foods that are more commonly savory to create desserts.

In [ ]:
fig = make_subplots(rows = 2, cols = 2, specs = [[{'type':'domain'},{'type':'domain'}],[{'type':'domain'},{'type':'domain'}]])
fig.add_trace(go.Pie(labels = wet_set['course'].value_counts().index, values =wet_set['course'].value_counts().values,pull=[0.1, 0, 0, 0], name = 'West india course profile'),1,1)
fig.add_trace(go.Pie(labels = east_set['course'].value_counts().index, values =east_set['course'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'East india course profile'),1,2)
fig.add_trace(go.Pie(labels = north_set['course'].value_counts().index, values =north_set['course'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'North india course profile'),2,1)
fig.add_trace(go.Pie(labels = south_set['course'].value_counts().index, values =south_set['course'].value_counts().values,pull=[0.1, 0, 0.1, 0.18], name = 'South india course profile'),2,2)
fig.update_traces(hoverinfo="label+percent+name")
fig.update_layout(
    title_text="Region wide course profile of India",
    annotations=[dict(text='West', x=0.186, y=0.5, font_size=20, showarrow=False),
                 dict(text='North', x=0.186, y=-0.12, font_size=20, showarrow=False),
                 dict(text='East', x=0.82, y=0.5, font_size=20, showarrow=False),
                 dict(text='South', x=0.82, y=-0.12, font_size=20, showarrow=False)])
fig.show()

### Flavor Profiles

Sweetness balances everything. And everyone loves it. It’s the degree to which sugars are detectable in foods, specifically glucose, lactose, and fructose. We, as animals, are scientifically geared to recognize sugar as a good thing, but foods with sugar (before pre-packaged stuff and bread) contain high carbohydrates and nutrients—like fruit.
Adding a touch of sweetness to a dish adds interest. It also balances spicy and sour tastes. Sweet and sour chicken, anyone? Stevia, maple syrup, carrots, balsamic vinegar, and most fruit are good sweetening agents to add to your own cooking.

The reason **spiciness** doesn’t qualify as a taste is because of the way we identify it in our food. In the case of the five tastes, our tongue has specific sensory cells that can detect them, and they have a specific nerve path. When things are spicy, we recognize it because it actually hurts our tongue. A pain signal is then sent to our brain by the nerves that transmit touch and temperature sensations.That being said, spiciness is a wonderful tool in the kitchen, and it’s still part of what makes up our sense of flavor. Spicy things are easy to identify—hot sauces, wasabi, horseradish, jalapeños, arugula, raw radishes. They are often used as enhancers, but don’t worry too much. If you like it spicy, just make it spicy.

Bitter is everyone’s least favorite taste, and we usually try to avoid it. Initially, it was an evolutionary defense—bitter plants were often the species of plants that were poisonous. Actually, there are more proteins in our sensory cells that respond to bitter taste than any other taste (we’re really good at protecting ourselves.)
But now, in our modern world, we can pair it alongside the other tastes to add depth to flavor profiles. It’s a good counterpoint for sweet and savory foods. You can use foods like coffee, spinach, kale, grapefruit juice, beer, endives, dandelion greens, broccoli, or radicchio to achieve bitterness.
To balance the bitterness, which is the more common pursuit, use sweet, sour, and salty foods. For instance, naturally bitter greens taste divine with vinaigrette, and a dollop of Greek yogurt is the perfect addition to meat.

Sour is how much acid you can detect in food. Adding sourness to food will balance spicy and sweetness. It’s really good if you want to cut through rich foods, especially ones that are high in fat. Sour things are things like lemon juice, lime juice, orange juice, a lot of the vinegar (sherry, red, balsamic, apple cider), tomato paste, yogurt, and sour cream.

In [ ]:
fig = make_subplots(rows = 2, cols = 2, specs = [[{'type':'domain'},{'type':'domain'}],[{'type':'domain'},{'type':'domain'}]])
fig.add_trace(go.Pie(labels = wet_set['flavor_profile'].value_counts().index, values =wet_set['flavor_profile'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'West india flavor profile'),1,1)
fig.add_trace(go.Pie(labels = east_set['flavor_profile'].value_counts().index, values =east_set['flavor_profile'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'East india flavor profile'),1,2)
fig.add_trace(go.Pie(labels = north_set['flavor_profile'].value_counts().index, values =north_set['flavor_profile'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'North india flavor profile'),2,1)
fig.add_trace(go.Pie(labels = south_set['flavor_profile'].value_counts().index, values =south_set['flavor_profile'].value_counts().values,pull=[0.1,0, 0, 0.18], name = 'South india flavor profile'),2,2)
fig.update_traces(hoverinfo="label+percent+name")
fig.update_layout(
    title_text="Region wide flavor profile of India",
    annotations=[dict(text='West', x=0.186, y=0.5, font_size=20, showarrow=False),
                 dict(text='North', x=0.186, y=-0.12, font_size=20, showarrow=False),
                 dict(text='East', x=0.795, y=0.5, font_size=20, showarrow=False),
                 dict(text='South', x=0.815, y=-0.12, font_size=20, showarrow=False)])
fig.show()

In [ ]:
fig = make_subplots(rows = 2, cols = 2, specs = [[{'type':'domain'},{'type':'domain'}],[{'type':'domain'},{'type':'domain'}]])
fig.add_trace(go.Pie(labels = wet_set['diet'].value_counts().index, values =wet_set['diet'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'West india diet profile'),1,1)
fig.add_trace(go.Pie(labels = east_set['diet'].value_counts().index, values =east_set['diet'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'East india diet profile'),1,2)
fig.add_trace(go.Pie(labels = north_set['diet'].value_counts().index, values =north_set['diet'].value_counts().values,pull=[0.1, 0.2, 0.15, 0.18], name = 'North india diet profile'),2,1)
fig.add_trace(go.Pie(labels = south_set['diet'].value_counts().index, values =south_set['diet'].value_counts().values,pull=[0.1,0, 0, 0.18], name = 'South india diet profile'),2,2)
fig.update_traces(hoverinfo="label+percent+name")
fig.update_layout(
    title_text="Region wide diet of India",
    annotations=[dict(text='West', x=0.186, y=0.5, font_size=20, showarrow=False),
                 dict(text='North', x=0.186, y=-0.12, font_size=20, showarrow=False),
                 dict(text='East', x=0.795, y=0.5, font_size=20, showarrow=False),
                 dict(text='South', x=0.815, y=-0.12, font_size=20, showarrow=False)])
fig.show()